# ⚡ Formats de Données

**Badge:** ⚡ Intermédiaire | ⏱ 45 min | 🔑 **Concepts clés :** CSV, JSON, Parquet, ligne vs colonne, partitionnement

## Objectifs

- Comprendre les différences entre formats ligne et colonne
- Maîtriser CSV et ses limitations
- Manipuler JSON et JSON Lines
- Découvrir Parquet et ses avantages
- Comparer les performances et tailles de fichiers
- Appliquer le partitionnement pour organiser les données

## Prérequis

- Pandas pour la manipulation de données
- Notions de systèmes de fichiers
- Bases de compression

## 1. Formats ligne vs colonne : Concept fondamental

### Format ligne (Row-oriented)

Stocke les données ligne par ligne. Chaque ligne contient tous les champs.

```
Row 1: [id=1, name="Alice", age=30, city="Paris"]
Row 2: [id=2, name="Bob", age=25, city="Lyon"]
Row 3: [id=3, name="Charlie", age=35, city="Nice"]
```

**Avantages** : 
- Efficace pour lire des lignes entières
- Insert/Update rapides
- Formats : CSV, JSON, JSONL

**Cas d'usage** : OLTP (transactions), lecture de toutes les colonnes

### Format colonne (Column-oriented)

Stocke les données colonne par colonne. Toutes les valeurs d'une colonne sont regroupées.

```
Column id:   [1, 2, 3]
Column name: ["Alice", "Bob", "Charlie"]
Column age:  [30, 25, 35]
Column city: ["Paris", "Lyon", "Nice"]
```

**Avantages** :
- Compression excellente (valeurs similaires)
- Lecture sélective de colonnes (I/O réduit)
- Agrégations rapides
- Formats : Parquet, ORC, Arrow

**Cas d'usage** : OLAP (analytique), data warehouses, Big Data

## 2. CSV : Le format universel

### Avantages
- Lisible par les humains
- Universel (Excel, SQL, tous les langages)
- Simple

### Inconvénients
- Pas de types de données (tout est string)
- Volumineux (texte non compressé)
- Lent pour gros volumes
- Problèmes d'encodage
- Pas de metadata

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import time

# Créer un dataset e-commerce
np.random.seed(42)

n_rows = 100_000
df_sales = pd.DataFrame({
    'order_id': range(1, n_rows + 1),
    'order_date': pd.date_range('2024-01-01', periods=n_rows, freq='5min'),
    'customer_id': np.random.randint(1, 10000, n_rows),
    'product_name': np.random.choice(['Laptop', 'Smartphone', 'Tablet', 'Headphones', 'Monitor'], n_rows),
    'category': np.random.choice(['Electronics', 'Accessories'], n_rows),
    'quantity': np.random.randint(1, 10, n_rows),
    'unit_price': np.random.uniform(10, 2000, n_rows).round(2),
    'discount': np.random.uniform(0, 0.3, n_rows).round(2),
    'total_amount': 0.0
})

# Calculer le montant total
df_sales['total_amount'] = (df_sales['quantity'] * df_sales['unit_price'] * (1 - df_sales['discount'])).round(2)

print(f"Dataset créé : {len(df_sales):,} lignes")
print(f"Colonnes : {list(df_sales.columns)}")
print(f"\nAperçu :")
print(df_sales.head())
print(f"\nMémoire : {df_sales.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Sauvegarder en CSV
Path('data_formats').mkdir(exist_ok=True)

csv_path = 'data_formats/sales.csv'

start = time.time()
df_sales.to_csv(csv_path, index=False)
write_time = time.time() - start

# Taille du fichier
file_size = Path(csv_path).stat().st_size / 1024**2

print(f"✓ CSV sauvegardé : {csv_path}")
print(f"  Temps écriture : {write_time:.3f}s")
print(f"  Taille : {file_size:.2f} MB")

# Lecture
start = time.time()
df_csv = pd.read_csv(csv_path, parse_dates=['order_date'])
read_time = time.time() - start

print(f"\n✓ CSV lu : {len(df_csv):,} lignes")
print(f"  Temps lecture : {read_time:.3f}s")

### Problèmes courants avec CSV

In [ ]:
# Problème 1 : Perte de types
print("Types après lecture CSV :")
print(df_csv.dtypes)
print("\n⚠️ Tous les types doivent être réinférés ou spécifiés manuellement")

# Problème 2 : Virgules dans les données
df_problematic = pd.DataFrame({
    'name': ['Alice, Bob', 'Charlie"s Store'],
    'description': ['Product with, comma', 'Line 1\nLine 2']
})

df_problematic.to_csv('data_formats/problematic.csv', index=False)
print("\n✓ CSV avec caractères spéciaux sauvegardé (virgules, quotes, newlines)")

# Problème 3 : Encodage
df_encoding = pd.DataFrame({
    'text': ['Café', 'naïve', '北京']
})

df_encoding.to_csv('data_formats/encoding.csv', index=False, encoding='utf-8')
print("✓ CSV avec encodage UTF-8 sauvegardé")
print("⚠️ Spécifiez toujours l'encodage : encoding='utf-8'")

## 3. JSON : Format structuré

### Avantages
- Structure hiérarchique (objets imbriqués)
- Types de données (nombres, booléens, null)
- Lisible par les humains
- Standard web (APIs)

### Inconvénients
- Verbeux (répétition des clés)
- Lent pour gros volumes
- Pas optimisé pour l'analytique

In [ ]:
# Sauvegarder en JSON
json_path = 'data_formats/sales.json'

start = time.time()
df_sales.to_json(json_path, orient='records', date_format='iso', indent=2)
json_write_time = time.time() - start

json_size = Path(json_path).stat().st_size / 1024**2

print(f"✓ JSON sauvegardé : {json_path}")
print(f"  Temps écriture : {json_write_time:.3f}s")
print(f"  Taille : {json_size:.2f} MB")

# Lecture
start = time.time()
df_json = pd.read_json(json_path)
json_read_time = time.time() - start

print(f"\n✓ JSON lu : {len(df_json):,} lignes")
print(f"  Temps lecture : {json_read_time:.3f}s")

# Aperçu du contenu JSON
with open(json_path, 'r') as f:
    sample = f.read(500)
    print(f"\nAperçu JSON (premiers caractères) :")
    print(sample)

### JSON Lines (JSONL) : Une ligne = un JSON

Format idéal pour les logs et le streaming.

In [ ]:
# JSON Lines : un JSON par ligne (pas de tableau global)
jsonl_path = 'data_formats/sales.jsonl'

start = time.time()
df_sales.to_json(jsonl_path, orient='records', lines=True, date_format='iso')
jsonl_write_time = time.time() - start

jsonl_size = Path(jsonl_path).stat().st_size / 1024**2

print(f"✓ JSONL sauvegardé : {jsonl_path}")
print(f"  Temps écriture : {jsonl_write_time:.3f}s")
print(f"  Taille : {jsonl_size:.2f} MB")

# Lecture
start = time.time()
df_jsonl = pd.read_json(jsonl_path, lines=True)
jsonl_read_time = time.time() - start

print(f"\n✓ JSONL lu : {len(df_jsonl):,} lignes")
print(f"  Temps lecture : {jsonl_read_time:.3f}s")

# Aperçu JSONL
with open(jsonl_path, 'r') as f:
    lines = [f.readline() for _ in range(3)]
    print("\nAperçu JSONL (3 premières lignes) :")
    for i, line in enumerate(lines, 1):
        print(f"  Ligne {i}: {line[:80]}...")

## 4. Parquet : Format colonnaire moderne

Parquet est le format de référence pour le Big Data et l'analytique.

### Avantages
- **Compression** excellente (5-10x plus petit que CSV)
- **Types de données** préservés
- **Lecture sélective** de colonnes (column pruning)
- **Rapide** pour l'analytique
- **Metadata** intégrée (schema, statistiques)
- **Compatible** Spark, Dask, BigQuery, Snowflake

### Inconvénients
- Binaire (pas lisible humainement)
- Nécessite une bibliothèque (pyarrow, fastparquet)

In [ ]:
# Installation : pip install pyarrow
import pyarrow.parquet as pq

# Sauvegarder en Parquet
parquet_path = 'data_formats/sales.parquet'

start = time.time()
df_sales.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)
parquet_write_time = time.time() - start

parquet_size = Path(parquet_path).stat().st_size / 1024**2

print(f"✓ Parquet sauvegardé : {parquet_path}")
print(f"  Temps écriture : {parquet_write_time:.3f}s")
print(f"  Taille : {parquet_size:.2f} MB")

# Lecture complète
start = time.time()
df_parquet = pd.read_parquet(parquet_path, engine='pyarrow')
parquet_read_time = time.time() - start

print(f"\n✓ Parquet lu : {len(df_parquet):,} lignes")
print(f"  Temps lecture : {parquet_read_time:.3f}s")
print(f"\nTypes préservés :")
print(df_parquet.dtypes)

### Lecture sélective de colonnes : Feature killer de Parquet

In [ ]:
# Lire seulement certaines colonnes (column pruning)
selected_columns = ['order_id', 'order_date', 'total_amount']

start = time.time()
df_partial = pd.read_parquet(parquet_path, columns=selected_columns)
partial_read_time = time.time() - start

print(f"✓ Lecture sélective : {len(df_partial):,} lignes, {len(df_partial.columns)} colonnes")
print(f"  Temps : {partial_read_time:.3f}s")
print(f"  Gain : {parquet_read_time / partial_read_time:.1f}x plus rapide")

print(f"\nMémoire :")
print(f"  Lecture complète : {df_parquet.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  Lecture sélective : {df_partial.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n✓ Avec Parquet, on ne lit que les colonnes nécessaires !")

### Metadata du fichier Parquet

In [ ]:
# Inspecter les metadata Parquet
parquet_file = pq.ParquetFile(parquet_path)

print("Metadata Parquet :")
print(f"  Nombre de lignes : {parquet_file.metadata.num_rows:,}")
print(f"  Nombre de row groups : {parquet_file.metadata.num_row_groups}")
print(f"  Schéma :")
print(parquet_file.schema)

# Statistiques par colonne
print("\nStatistiques (premier row group) :")
rg = parquet_file.metadata.row_group(0)
for i in range(min(3, rg.num_columns)):
    col = rg.column(i)
    print(f"  {col.path_in_schema}: {col.statistics}")

## 5. Comparaison des formats : Performance et taille

In [ ]:
import matplotlib.pyplot as plt

# Tableau comparatif
comparison = pd.DataFrame({
    'Format': ['CSV', 'JSON', 'JSONL', 'Parquet'],
    'Taille (MB)': [file_size, json_size, jsonl_size, parquet_size],
    'Ecriture (s)': [write_time, json_write_time, jsonl_write_time, parquet_write_time],
    'Lecture (s)': [read_time, json_read_time, jsonl_read_time, parquet_read_time]
})

comparison['Ratio taille vs CSV'] = (comparison['Taille (MB)'] / file_size).round(2)

print("COMPARAISON DES FORMATS")
print("="*70)
print(comparison.to_string(index=False))

print(f"\n💡 Parquet est {file_size / parquet_size:.1f}x plus compact que CSV")
print(f"💡 Parquet est {read_time / parquet_read_time:.1f}x plus rapide à lire que CSV")

In [ ]:
# Graphiques de comparaison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Graphique 1 : Taille
axes[0].bar(comparison['Format'], comparison['Taille (MB)'], color=['blue', 'orange', 'green', 'red'])
axes[0].set_ylabel('Taille (MB)')
axes[0].set_title('Taille des fichiers')
axes[0].tick_params(axis='x', rotation=45)

# Graphique 2 : Ecriture
axes[1].bar(comparison['Format'], comparison['Ecriture (s)'], color=['blue', 'orange', 'green', 'red'])
axes[1].set_ylabel('Temps (s)')
axes[1].set_title('Temps d\'écriture')
axes[1].tick_params(axis='x', rotation=45)

# Graphique 3 : Lecture
axes[2].bar(comparison['Format'], comparison['Lecture (s)'], color=['blue', 'orange', 'green', 'red'])
axes[2].set_ylabel('Temps (s)')
axes[2].set_title('Temps de lecture')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n✓ Parquet gagne sur tous les tableaux pour l'analytique !")

## 6. Partitionnement : Organiser les données

Le **partitionnement** consiste à diviser les données en sous-dossiers selon une ou plusieurs colonnes.

### Avantages
- Lecture sélective (skip des partitions inutiles)
- Parallélisation facile
- Organisation logique

### Pattern courant
```
data/
  year=2024/
    month=01/
      part-0.parquet
      part-1.parquet
    month=02/
      part-0.parquet
```

In [ ]:
# Ajouter des colonnes de partitionnement
df_sales['year'] = df_sales['order_date'].dt.year
df_sales['month'] = df_sales['order_date'].dt.month

print(f"Dataset avec colonnes de partitionnement :")
print(df_sales[['order_date', 'year', 'month', 'total_amount']].head())

# Sauvegarder avec partitionnement
partitioned_path = 'data_formats/sales_partitioned'

start = time.time()
df_sales.to_parquet(
    partitioned_path,
    engine='pyarrow',
    partition_cols=['year', 'month'],  # Partitionner par année et mois
    index=False
)
print(f"\n✓ Données partitionnées sauvegardées en {time.time() - start:.3f}s")

# Structure des dossiers
import os
print("\nStructure des dossiers :")
for root, dirs, files in os.walk(partitioned_path):
    level = root.replace(partitioned_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:2]:  # Limite à 2 fichiers par dossier
        print(f"{subindent}{file}")
    if len(files) > 2:
        print(f"{subindent}... (+{len(files)-2} fichiers)")

In [ ]:
# Lecture sélective avec filtres sur partitions
start = time.time()
df_filtered = pd.read_parquet(
    partitioned_path,
    filters=[('year', '=', 2024), ('month', '=', 1)]  # Lire seulement janvier 2024
)
filter_time = time.time() - start

print(f"✓ Lecture filtrée (janvier 2024) : {len(df_filtered):,} lignes")
print(f"  Temps : {filter_time:.3f}s")
print(f"\n💡 Seule la partition year=2024/month=1 a été lue !")

# Lecture complète pour comparaison
start = time.time()
df_full = pd.read_parquet(partitioned_path)
full_time = time.time() - start

print(f"\n✓ Lecture complète : {len(df_full):,} lignes")
print(f"  Temps : {full_time:.3f}s")
print(f"\n  Gain avec filtrage : {full_time / filter_time:.1f}x plus rapide")

## 7. Autres formats (mention pour culture)

### Avro
- Format ligne binaire avec schema
- Populaire dans Kafka
- Bon pour streaming et évolution de schema

### ORC (Optimized Row Columnar)
- Format colonnaire similaire à Parquet
- Optimisé pour Hadoop/Hive
- Compression légèrement meilleure que Parquet

### Apache Arrow
- Format en mémoire (pas pour le stockage)
- Échange zero-copy entre langages (Python, R, Java)
- Base de Parquet et de nombreux outils modernes

## 8. Lien avec Spark et le Big Data

Les formats colonnaires (Parquet, ORC) sont essentiels dans l'écosystème Big Data :

- **Spark** : format par défaut pour le stockage
- **Hive** : tables externes en Parquet
- **BigQuery** : import/export Parquet natif
- **Snowflake** : lecture directe de Parquet sur S3
- **Dask** : partitionnement automatique en Parquet

**Philosophie** : même format du laptop au data lake, scalabilité naturelle.

## Pièges courants

### 1. Encodage CSV

In [ ]:
# ❌ Oublier l'encodage
# df.to_csv('file.csv')  # Peut causer des erreurs avec accents

# ✅ Toujours spécifier UTF-8
df_encoding = pd.DataFrame({'text': ['Café', 'naïve', '北京']})
df_encoding.to_csv('data_formats/test_encoding.csv', index=False, encoding='utf-8')
df_read = pd.read_csv('data_formats/test_encoding.csv', encoding='utf-8')
print("✓ Encodage UTF-8 correct")
print(df_read)

### 2. Dates en JSON

In [ ]:
# ❌ Dates converties en timestamps ou strings
df_dates = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=3)
})

# Sauvegarder avec format ISO
df_dates.to_json('data_formats/dates.json', orient='records', date_format='iso')

# ✅ Relire avec parse_dates si nécessaire
df_dates_read = pd.read_json('data_formats/dates.json')
print("Type après lecture JSON :")
print(df_dates_read.dtypes)
print("\n✓ Avec Parquet, les dates sont préservées automatiquement !")

### 3. Version et compression Parquet

In [ ]:
# Tester différentes compressions
compressions = ['snappy', 'gzip', 'brotli', 'none']
results = []

for comp in compressions:
    path = f'data_formats/sales_{comp}.parquet'
    
    try:
        start = time.time()
        df_sales.to_parquet(path, compression=comp, index=False)
        write_t = time.time() - start
        
        size = Path(path).stat().st_size / 1024**2
        
        start = time.time()
        _ = pd.read_parquet(path)
        read_t = time.time() - start
        
        results.append({
            'Compression': comp,
            'Taille (MB)': round(size, 2),
            'Ecriture (s)': round(write_t, 3),
            'Lecture (s)': round(read_t, 3)
        })
    except Exception as e:
        print(f"❌ {comp} non disponible")

df_compressions = pd.DataFrame(results)
print("\nComparaison des compressions Parquet :")
print(df_compressions.to_string(index=False))

print("\n💡 Snappy = bon compromis (vitesse + compression)")
print("💡 Gzip = meilleure compression, plus lent")

## Mini-exercices

### Exercice 1 : Convertir CSV → Parquet

1. Créez un CSV de 50k lignes avec colonnes : id, name, age, city, salary  
2. Chargez-le dans Pandas  
3. Sauvegardez-le en Parquet avec compression snappy  
4. Comparez les tailles et temps de lecture

In [ ]:
# Votre code ici


### Exercice 2 : Lecture sélective de colonnes

À partir du fichier Parquet `sales.parquet` :  
1. Lisez uniquement les colonnes : order_id, customer_id, total_amount  
2. Mesurez le temps et la mémoire utilisée  
3. Comparez avec une lecture complète

In [ ]:
# Votre code ici


### Exercice 3 : Partitionnement par catégorie

1. Créez un DataFrame avec 10k ventes, 5 catégories de produits, dates sur 6 mois  
2. Sauvegardez en Parquet partitionné par catégorie et mois  
3. Lisez uniquement la catégorie "Electronics" du mois 3  
4. Vérifiez que seules les bonnes partitions ont été lues

In [ ]:
# Votre code ici


## Solutions des exercices

In [ ]:
# Solution Exercice 1
n = 50_000
df_ex1 = pd.DataFrame({
    'id': range(1, n+1),
    'name': [f'Person_{i}' for i in range(n)],
    'age': np.random.randint(18, 70, n),
    'city': np.random.choice(['Paris', 'Lyon', 'Marseille', 'Toulouse'], n),
    'salary': np.random.uniform(25000, 100000, n).round(2)
})

# CSV
csv_ex1 = 'data_formats/ex1.csv'
start = time.time()
df_ex1.to_csv(csv_ex1, index=False)
csv_write = time.time() - start
csv_size = Path(csv_ex1).stat().st_size / 1024**2

start = time.time()
df_csv_ex1 = pd.read_csv(csv_ex1)
csv_read = time.time() - start

# Parquet
parquet_ex1 = 'data_formats/ex1.parquet'
start = time.time()
df_ex1.to_parquet(parquet_ex1, compression='snappy', index=False)
parquet_write = time.time() - start
parquet_size = Path(parquet_ex1).stat().st_size / 1024**2

start = time.time()
df_parquet_ex1 = pd.read_parquet(parquet_ex1)
parquet_read = time.time() - start

print("COMPARAISON CSV vs PARQUET")
print(f"\nCSV :")
print(f"  Taille : {csv_size:.2f} MB")
print(f"  Lecture : {csv_read:.3f}s")
print(f"\nParquet :")
print(f"  Taille : {parquet_size:.2f} MB (gain {csv_size/parquet_size:.1f}x)")
print(f"  Lecture : {parquet_read:.3f}s (gain {csv_read/parquet_read:.1f}x)")

In [ ]:
# Solution Exercice 2
selected_cols = ['order_id', 'customer_id', 'total_amount']

# Lecture complète
start = time.time()
df_full_ex2 = pd.read_parquet(parquet_path)
full_time_ex2 = time.time() - start
full_memory = df_full_ex2.memory_usage(deep=True).sum() / 1024**2

# Lecture sélective
start = time.time()
df_partial_ex2 = pd.read_parquet(parquet_path, columns=selected_cols)
partial_time_ex2 = time.time() - start
partial_memory = df_partial_ex2.memory_usage(deep=True).sum() / 1024**2

print("LECTURE SÉLECTIVE vs COMPLÈTE")
print(f"\nComplète ({len(df_full_ex2.columns)} colonnes) :")
print(f"  Temps : {full_time_ex2:.3f}s")
print(f"  Mémoire : {full_memory:.2f} MB")
print(f"\nSélective ({len(selected_cols)} colonnes) :")
print(f"  Temps : {partial_time_ex2:.3f}s (gain {full_time_ex2/partial_time_ex2:.1f}x)")
print(f"  Mémoire : {partial_memory:.2f} MB (économie {full_memory/partial_memory:.1f}x)")
print(f"\n✓ Ne lisez que ce dont vous avez besoin !")

In [ ]:
# Solution Exercice 3
n_ex3 = 10_000
categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Toys']

df_ex3 = pd.DataFrame({
    'sale_id': range(1, n_ex3+1),
    'date': pd.date_range('2024-01-01', periods=n_ex3, freq='2h'),
    'category': np.random.choice(categories, n_ex3),
    'amount': np.random.uniform(10, 500, n_ex3).round(2),
    'quantity': np.random.randint(1, 10, n_ex3)
})

df_ex3['month'] = df_ex3['date'].dt.month

# Sauvegarder partitionné
partition_path_ex3 = 'data_formats/sales_by_category'
df_ex3.to_parquet(
    partition_path_ex3,
    partition_cols=['category', 'month'],
    index=False
)
print("✓ Données partitionnées par catégorie et mois")

# Lire seulement Electronics mois 3
start = time.time()
df_filtered_ex3 = pd.read_parquet(
    partition_path_ex3,
    filters=[
        ('category', '=', 'Electronics'),
        ('month', '=', 3)
    ]
)
filter_time_ex3 = time.time() - start

print(f"\n✓ Lecture filtrée : {len(df_filtered_ex3)} lignes")
print(f"  Temps : {filter_time_ex3:.3f}s")
print(f"\n  Catégories dans résultat : {df_filtered_ex3['category'].unique()}")
print(f"  Mois dans résultat : {df_filtered_ex3['month'].unique()}")
print("\n✓ Seule la partition category=Electronics/month=3 a été lue !")

## Résumé

### Points clés

1. **Format ligne** (CSV, JSON) : bon pour transactions, insertion ligne par ligne
2. **Format colonne** (Parquet) : excellent pour analytique, compression, lecture sélective
3. **CSV** : universel mais lent, volumineux, perte de types
4. **JSON** : structuré, verbeux, bon pour APIs
5. **Parquet** : format de référence pour Big Data, 5-10x plus compact que CSV
6. **Lecture sélective** : avec Parquet, ne lisez que les colonnes nécessaires
7. **Partitionnement** : organisez vos données par date/catégorie pour des lectures ultra-rapides

### Règles de décision

| Cas d'usage | Format recommandé |
|-------------|-------------------|
| Export Excel, lisibilité humaine | CSV |
| API REST, configuration | JSON |
| Logs, streaming | JSON Lines |
| Data lake, analytique | Parquet partitionné |
| Data warehouse | Parquet ou ORC |
| Kafka, messaging | Avro |

### Prochaines étapes

- Notebook suivant : **DuckDB**
- Approfondir : Spark, Delta Lake, Iceberg